# Hyperparameter Tuning - GRU4Rec

Este notebook implementa busca de hiperparâmetros usando RecBole's HyperTuning.

Referência: https://github.com/RUCAIBox/RecBole/blob/master/docs/source/user_guide/usage/parameter_tuning.rst

## 1. Setup e Importações

In [ ]:
import sys
import os
from pathlib import Path

# Add project root to path
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
sys.path.insert(0, str(PROJECT_ROOT))

print(f"Project root: {PROJECT_ROOT}")

In [ ]:
import logging
import numpy as np
import torch
from recbole.trainer import HyperTuning
from recbole.quick_start import objective_function

logging.basicConfig(level=logging.INFO)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 2. Configuração Base

In [ ]:
# Configurações do projeto
MODEL_NAME = 'GRU4Rec'
DATASET_NAME = 'realestate_simple'
GPU_ID = 0

# Caminhos
DATA_PATH = PROJECT_ROOT / 'outputs' / 'data' / 'recbole_simple'
CHECKPOINT_DIR = PROJECT_ROOT / 'outputs' / 'tuning'
CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Data path: {DATA_PATH}")
print(f"Checkpoint dir: {CHECKPOINT_DIR}")

## 3. Configuração de Hiperparâmetros

Define o espaço de busca e parâmetros fixos.

In [ ]:
# Parâmetros fixos (não serão otimizados)
config_dict = {
    'model': MODEL_NAME,
    'dataset': DATASET_NAME,
    'data_path': str(DATA_PATH),
    
    # Training settings
    'epochs': 10,
    'train_batch_size': 2048,
    'eval_batch_size': 2048,
    'train_neg_sample_args': None,
    
    # Early Stopping
    'stopping_step': 2,
    
    # Fixed Model Parameters
    'loss_type': 'CE',
    'num_layers': 2,
    
    # Gradient Clipping
    'clip_grad_norm': {'max_norm': 5.0},
    
    # Evaluation
    'metrics': ['Recall', 'MRR', 'NDCG', 'Hit'],
    'topk': [5, 10, 20],
    'valid_metric': 'MRR@10',
    
    # Session Settings
    'MAX_ITEM_LIST_LENGTH': 50,
    'SESSION_ID_FIELD': 'session_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'USER_ID_FIELD': 'session_id',
    'load_col': {'inter': ['session_id', 'item_id', 'timestamp']},
    
    # Evaluation Protocol
    'eval_args': {
        'split': {'LS': 'valid_and_test'},
        'order': 'TO',
        'mode': 'full'
    },
    
    # Device
    'device': 'cuda',
    'gpu_id': GPU_ID,
    
    # Reproducibility
    'seed': 42,
    
    # Checkpointing
    'save_dataset': False,
    'save_dataloaders': False,
    'checkpoint_dir': str(CHECKPOINT_DIR),
    
    # Logging
    'log_wandb': False,
}

print("Fixed configuration loaded")

In [ ]:
# Espaço de busca de hiperparâmetros
# Formato: {'param_name': 'choice/uniform/loguniform/[values]'}

hyperparameter_space = {
    # Learning rate (log scale)
    'learning_rate': 'loguniform(1e-4, 1e-2)',
    
    # Embedding dimension
    'embedding_size': 'choice([32, 64, 128])',
    
    # Hidden dimension
    'hidden_size': 'choice([64, 128, 256])',
    
    # Dropout probability
    'dropout_prob': 'uniform(0.1, 0.5)',
}

print("\nHyperparameter search space:")
for param, space in hyperparameter_space.items():
    print(f"  {param}: {space}")

## 4. Configuração do HyperTuning

RecBole suporta vários métodos de otimização:
- **loguniform**: busca em escala logarítmica (ex: learning rate)
- **uniform**: busca em escala linear (ex: dropout)
- **choice**: escolha discreta (ex: dimensões)
- **randint**: inteiros aleatórios

In [ ]:
# Configuração do tuning
hp = HyperTuning(
    objective_function=objective_function,
    algo='exhaustive',  # Options: 'exhaustive', 'random', 'bayes'
    early_stop=10,      # Stop if no improvement after N trials
    max_evals=20,       # Maximum number of evaluations
    params_dict=config_dict,
    params_file=None,
    fixed_config_file_list=[],
)

print("HyperTuning initialized:")
print(f"  Algorithm: exhaustive")
print(f"  Max evaluations: 20")
print(f"  Early stop: 10 trials without improvement")

## 5. Executar Busca de Hiperparâmetros

⚠️ **Atenção**: Isso pode levar várias horas dependendo do número de trials.

In [ ]:
# Executar tuning
best_result, best_params = hp.run(
    space=hyperparameter_space,
    verbose=True,
    show_progress=True
)

print("\n" + "="*80)
print("HYPERPARAMETER TUNING COMPLETED")
print("="*80)

## 6. Melhores Resultados

In [ ]:
print("\n🏆 BEST HYPERPARAMETERS:")
print("="*60)
for param, value in best_params.items():
    print(f"  {param}: {value}")

print("\n📊 BEST RESULTS:")
print("="*60)
for metric, value in best_result.items():
    print(f"  {metric}: {value:.4f}")

## 7. Salvar Melhores Hiperparâmetros

In [ ]:
import json
from datetime import datetime

# Save best parameters
output_file = CHECKPOINT_DIR / f'best_params_{MODEL_NAME}_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json'

results = {
    'model': MODEL_NAME,
    'dataset': DATASET_NAME,
    'timestamp': datetime.now().isoformat(),
    'best_params': best_params,
    'best_results': best_result,
    'search_space': hyperparameter_space,
}

with open(output_file, 'w') as f:
    json.dump(results, f, indent=2, default=str)

print(f"✅ Best parameters saved to: {output_file}")

## 8. Treinar Modelo Final com Melhores Hiperparâmetros

In [ ]:
from recbole.quick_start import run_recbole

# Update config with best params
final_config = config_dict.copy()
final_config.update(best_params)
final_config['checkpoint_dir'] = str(CHECKPOINT_DIR / 'final_model')

print("Training final model with best hyperparameters...")
print("\nFinal configuration:")
for key in ['learning_rate', 'embedding_size', 'hidden_size', 'dropout_prob']:
    if key in final_config:
        print(f"  {key}: {final_config[key]}")

In [ ]:
# Train final model
result = run_recbole(
    model=MODEL_NAME,
    dataset=DATASET_NAME,
    config_dict=final_config
)

print("\n" + "="*80)
print("FINAL MODEL TRAINING COMPLETED")
print("="*80)
print("\nTest Results:")
for metric, value in result.items():
    print(f"  {metric}: {value:.4f}")

## 9. Análise de Resultados

In [ ]:
# Compare baseline vs tuned
baseline_config = {
    'learning_rate': 0.001,
    'embedding_size': 64,
    'hidden_size': 128,
    'dropout_prob': 0.3,
}

print("\n📈 COMPARISON: Baseline vs Tuned")
print("="*60)
print("\nBaseline parameters:")
for param, value in baseline_config.items():
    best_value = best_params.get(param, 'N/A')
    arrow = '→' if param in best_params else ''
    print(f"  {param}: {value} {arrow} {best_value}")

print("\n💡 Insights:")
print("  - Check which parameters changed significantly")
print("  - Compare baseline vs tuned performance")
print("  - Consider computational cost vs performance gain")

## 10. Exportar para YAML (opcional)

In [ ]:
import yaml

# Create YAML config with best params
yaml_config = final_config.copy()

# Remove non-serializable items
yaml_config.pop('device', None)
yaml_config['data_path'] = 'outputs/data/recbole_simple'
yaml_config['checkpoint_dir'] = 'outputs/saved'

yaml_output = PROJECT_ROOT / 'src' / 'configs' / 'neural' / f'{MODEL_NAME.lower()}_tuned.yaml'

with open(yaml_output, 'w') as f:
    yaml.dump(yaml_config, f, default_flow_style=False, sort_keys=False)

print(f"\n✅ YAML config saved to: {yaml_output}")
print("\nYou can now use this config in your experiments!")